In [66]:
from statsmodels.stats.inter_rater import fleiss_kappa
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
import ast

In [5]:
def extract_number(filename):
    return int(filename.split('_')[-1].split('.')[0])

In [3]:
PATH = '../data'

In [7]:
LITBANK_INDEX=0

In [33]:
check_folder_path = '../data/NewLitBank_v2/NewLitBank_v2'
litbank_file_list = sorted(os.listdir(check_folder_path), key=extract_number)

In [62]:
df_case = pd.read_csv(check_folder_path + '/' + litbank_file_list[LITBANK_INDEX], index_col=0)
df_case['coref'] = [ast.literal_eval(data) for data in df_case['coref']]
df_case['extracted_sentence'] = [ast.literal_eval(data) for data in df_case['extracted_sentence']]
df_case['adjusted_offsets'] = [ast.literal_eval(data) for data in df_case['adjusted_offsets']]
df_case['text'] = [ast.literal_eval(data) for data in df_case['text']]
df_case['inference_offsets'] = [ast.literal_eval(data) for data in df_case['inference_offsets']]
df_case = df_case[df_case['update_text'].notna()].reset_index(drop=True)

In [ ]:
df_dh = pd.read_csv(PATH + '/NewLitBank_dh/' + litbank_file_list[LITBANK_INDEX], index_col=0)
df_js = pd.read_csv(PATH + '/NewLitBank_JS/' + litbank_file_list[LITBANK_INDEX], index_col=0).reset_index(drop=True)
df_gy = pd.read_csv(PATH + '/NewLitBank_gy/' + litbank_file_list[LITBANK_INDEX])

In [63]:
df_case

,coref,extracted_sentence,adjusted_offsets,text,inference_offsets,labels,update_text,update_sentence,update_coref
0,"[[450, 454], [18, 19], [22, 23], [247, 250], [...","[To, accommodate, the, regiments, of, miners, ...","[[35, 39], [64, 65], [68, 69], [90, 93], [116,...","[[the, site, of, Hell, Row], [Hell, Row], [Hel...","[[[7, 11], [41, 41]], [[43, 44], [58, 59]], [[...",0,"[['the', 'site', 'of', 'Hell', 'Row'], ['notor...","['To', 'accommodate', 'the', 'regiments', 'of'...","[[35, 39], [64, 66], [69, 71], [92, 95], [118,..."
1,"[[43, 53], [605, 606], [144, 145]]","[There, lived, the, colliers, who, worked, in,...","[[2, 12], [30, 31], [39, 40]]","[[the, colliers, who, worked, in, the, little,...",[],0,"[['the', 'colliers', 'who', 'worked', 'in', 't...","['There', 'lived', 'the', 'colliers', 'who', '...","[[2, 12], [30, 32], [40, 42]]"
2,"[[209, 209], [393, 393]]","[The, coal, and, iron, field, of, Nottinghamsh...","[[8, 8], [92, 92]]","[[Derbyshire], [Derbyshire]]","[[[50, 50], [59, 59]], [[67, 69], [73, 73]], [...",0,"[['Derbyshire'], ['picturesque', 'Derbyshire']]","['The', 'coal', 'and', 'iron', 'field', 'of', ...","[[8, 8], [92, 93]]"
3,"[[345, 346], [234, 235]]","[From, Nuttall, ,, high, up, on, the, sandston...","[[32, 33], [116, 117]]","[[Spinney, Park], [Spinney, Park]]","[[[39, 39], [48, 48]], [[56, 58], [62, 62]]]",0,"[['Spinney', 'picturesque', 'Park'], ['Spinney...","['From', 'Nuttall', ',', 'high', 'up', 'on', '...","[[32, 34], [117, 119]]"
4,"[[382, 382], [296, 296]]","[From, Nuttall, ,, high, up, on, the, sandston...","[[69, 69], [125, 125]]","[[Selby], [Selby]]","[[[38, 38], [47, 47]], [[55, 57], [61, 61]], [...",0,"[['Selby'], ['the', 'nearby', 'Selby']]","['From', 'Nuttall', ',', 'high', 'up', 'on', '...","[[69, 69], [125, 127]]"
5,"[[314, 314], [298, 298]]","[From, Nuttall, ,, high, up, on, the, sandston...","[[1, 1], [127, 127]]","[[Nuttall], [Nuttall]]","[[[38, 38], [47, 47]], [[55, 57], [61, 61]], [...",0,"[['Nuttall'], ['Nuttall', 'new']]","['From', 'Nuttall', ',', 'high', 'up', 'on', '...","[[1, 1], [127, 128]]"
6,"[[1162, 1162], [1183, 1183]]","[“, They, 'll, be, beginnin, ', ,, ”, the, boy...","[[1, 1], [22, 22]]","[[They], [they]]",[],0,"[['They'], ['the', 'mother']]","['“', 'They', ""'ll"", 'be', 'beginnin', ""'"", ',...","[[1, 1], [22, 23]]"


In [45]:
df_gy

,coref,extracted_sentence,adjusted_offsets,text,inference_offsets,labels,update_text,update_sentence,update_coref,cases
0,"[[450, 454], [18, 19], [22, 23], [247, 250], [...","['To', 'accommodate', 'the', 'regiments', 'of'...","[[35, 39], [64, 65], [68, 69], [90, 93], [116,...","[['the', 'site', 'of', 'Hell', 'Row'], ['Hell'...","[[[7, 11], [41, 41]], [[43, 44], [58, 59]], [[...",0,"[['the', 'site', 'of', 'Hell', 'Row'], ['notor...","['To', 'accommodate', 'the', 'regiments', 'of'...","[[35, 39], [64, 66], [69, 71], [92, 95], [118,...",0
1,"[[43, 53], [605, 606], [144, 145]]","['There', 'lived', 'the', 'colliers', 'who', '...","[[2, 12], [30, 31], [39, 40]]","[['the', 'colliers', 'who', 'worked', 'in', 't...",[],0,"[['the', 'colliers', 'who', 'worked', 'in', 't...","['There', 'lived', 'the', 'colliers', 'who', '...","[[2, 12], [30, 32], [40, 42]]",0
2,"[[209, 209], [393, 393]]","['The', 'coal', 'and', 'iron', 'field', 'of', ...","[[8, 8], [92, 92]]","[['Derbyshire'], ['Derbyshire']]","[[[50, 50], [59, 59]], [[67, 69], [73, 73]], [...",0,"[['Derbyshire'], ['picturesque', 'Derbyshire']]","['The', 'coal', 'and', 'iron', 'field', 'of', ...","[[8, 8], [92, 93]]",0
3,"[[345, 346], [234, 235]]","['From', 'Nuttall', ',', 'high', 'up', 'on', '...","[[32, 33], [116, 117]]","[['Spinney', 'Park'], ['Spinney', 'Park']]","[[[39, 39], [48, 48]], [[56, 58], [62, 62]]]",0,"[['Spinney', 'picturesque', 'Park'], ['Spinney...","['From', 'Nuttall', ',', 'high', 'up', 'on', '...","[[32, 34], [117, 119]]",1
4,"[[382, 382], [296, 296]]","['From', 'Nuttall', ',', 'high', 'up', 'on', '...","[[69, 69], [125, 125]]","[['Selby'], ['Selby']]","[[[38, 38], [47, 47]], [[55, 57], [61, 61]], [...",0,"[['Selby'], ['the', 'nearby', 'Selby']]","['From', 'Nuttall', ',', 'high', 'up', 'on', '...","[[69, 69], [125, 127]]",0
5,"[[314, 314], [298, 298]]","['From', 'Nuttall', ',', 'high', 'up', 'on', '...","[[1, 1], [127, 127]]","[['Nuttall'], ['Nuttall']]","[[[38, 38], [47, 47]], [[55, 57], [61, 61]], [...",0,"[['Nuttall'], ['Nuttall', 'new']]","['From', 'Nuttall', ',', 'high', 'up', 'on', '...","[[1, 1], [127, 128]]",1
6,"[[1162, 1162], [1183, 1183]]","['“', 'They', ""'ll"", 'be', 'beginnin', ""'"", ',...","[[1, 1], [22, 22]]","[['They'], ['they']]",[],0,"[['They'], ['the', 'mother']]","['“', 'They', ""'ll"", 'be', 'beginnin', ""'"", ',...","[[1, 1], [22, 23]]",2


In [50]:
num_classes = 3

In [64]:
def final_labels(count_matrix):
    argmax_labels = []
    for row in count_matrix:
        # 가장 큰 값의 인덱스들 (tie 가능성 고려)
        max_val = np.max(row)
        max_indices = np.where(row == max_val)[0]

        if len(max_indices) == 1:
            argmax_labels.append(max_indices[0])  # 유일한 최댓값이면 그 index
        else:
            argmax_labels.append(1)
    return argmax_labels

In [ ]:
total_count_matrix = []
for idx in tqdm(range(80)):
    df_case = pd.read_csv(check_folder_path + '/' + litbank_file_list[idx], index_col=0)
    df_case['coref'] = [ast.literal_eval(data) for data in df_case['coref']]
    df_case['extracted_sentence'] = [ast.literal_eval(data) for data in df_case['extracted_sentence']]
    df_case['adjusted_offsets'] = [ast.literal_eval(data) for data in df_case['adjusted_offsets']]
    df_case['text'] = [ast.literal_eval(data) for data in df_case['text']]
    df_case['inference_offsets'] = [ast.literal_eval(data) for data in df_case['inference_offsets']]
    df_case = df_case[df_case['update_text'].notna()].reset_index(drop=True)
    df_dh = pd.read_csv(PATH + '/NewLitBank_dh/' + litbank_file_list[idx], index_col=0)
    df_js = pd.read_csv(PATH + '/NewLitBank_JS/' + litbank_file_list[idx], index_col=0).reset_index(drop=True)
    df_gy = pd.read_csv(PATH + '/NewLitBank_gy/' + litbank_file_list[idx])
    annotators = [df_dh, df_js, df_gy]
    num_cases = len(df_dh)
    count_matrix = np.zeros((num_cases, num_classes), dtype=int)
    for df in annotators:
        if len(df['cases'])==0: 
            break
        for i, label in enumerate(df['cases']):
            count_matrix[i, label] += 1
    df_case['cases'] = final_labels(count_matrix)
    df_case.to_csv('../data/NewLitBank_ECAI/' + litbank_file_list[idx])
    total_count_matrix.append(count_matrix)

100%|██████████| 80/80 [00:01<00:00, 66.47it/s]


In [77]:
total_count_matrix

array([[1, 2, 0],
       [3, 0, 0],
       [3, 0, 0],
       ...,
       [1, 2, 0],
       [3, 0, 0],
       [1, 2, 0]])

In [76]:
total_count_matrix = np.vstack(total_count_matrix)

In [79]:
kappa = fleiss_kappa(total_count_matrix)

In [81]:
print(f"Fleiss' Kappa: {kappa:.4f}")

Fleiss' Kappa: 0.4630
